In [15]:
import platform
platform.python_version()

!pip install tflearn

In [25]:
!pip install tensorflow
!pip install tflearn

import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer= LancasterStemmer()
import numpy as np
import tflearn
import tensorflow as tf
import random
import json



ValueError: source code string cannot contain null bytes

In [ ]:
# import nltk
#nltk.download('punkt')

In [ ]:
with open("intents.json") as file:
    data=json.load(file)


In [ ]:
#print(data)
print(data["intents"])

In [ ]:
words=[]
labels=[]
docs_x=[]
docs_y=[]

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds=nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
    
    if intent["tag"] not in labels:
        labels.append(intent["tag"])
#print(docs_x)       
#print(docs_y)
#print(labels)
#print(len(words))
print(docs_x)

In [ ]:
#we now stem all words to remove duplicate and find vocabs
words=[stemmer.stem(w.lower()) for w in words if w!="?"]
words=sorted(list(set(words)))# set removes dups, list converts to list and sort sorts it
labels=sorted(labels)
print(words)

In [ ]:
#we create bag of words with one hot enceoder
#it determines if  a word exists or not, 
# eg [0,1,0,1,0,0,1,0,1,0,1,1,1,1,1,1,.....]

#words is the input data sent into the bag of words


#the output is the tags we have: eg: greeting, etc
training=[]
output=[]
out_empty=[0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag=[] #this reps the one hot encoded words 
    
    wrds=[stemmer.stem(w) for w in doc]# we stem again into docs_x
    
    for w in words:
        if w in wrds:
            bag.append(1) 
 #CODE ABOVE MEANS, the new wrds list created, if an object is
#in both the wrds list and the words list, append 1,
#it means the word exists in the current pattern we are loooping through
        else:
            bag.append(0)
  

    output_row=out_empty[:]
    output_row[labels.index(docs_y[x])] =1
    #we just loop through the labels and where the tag tag exists, we put one
    #into output and where it doesnt, we put zero
    
    
    training.append(bag)
    output.append(output_row)
    
training =np.array(training)
output=np.array(output)
print(len(training))
print(len(output))


In [ ]:
#pip install tflearn

In [ ]:
from tflearn import fully_connected

In [ ]:
#AI aspect

tensorflow.reset_default_graph()
net = tflearn.input_data(shape=[None, len(training)#input data
#this defines expected model input shape
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)# hidden layer with 8 neurons
#network = fully_connected(network, 10, activation='softmax')
#network = fully_connected(network, 10, activation='softmax')
                                
net=tflearn.fully_connected(net,len(output[0]), activation="softmax")#output
net=tflearn.regression(net)
                            
                            
                            
model=tflearn.DNN(net)                            
model.fit(training,output, n_epoch=1000, batch_size=8)
model.save("model.tflearn")